In [ ]:
from sage.algebras.flag_algebras import *

In [28]:
### This cell is just to set up the theory.
### In practice it is not needed, as the calculations (multiplication table and generated structures)
### are already done and saved. But it is here for completeness (and for re-runs from scratch)

# These are helper functions, to deal with classical exclusion (not just induced)
def check_containment(smalls, larges):
    sis = [IncidenceStructure(ss.size(), ss.blocks()['edges']) for ss in smalls]
    lis = [IncidenceStructure(ss.size(), ss.blocks()['edges']) for ss in larges]
    res = []
    for ll in lis:
        good = True
        for ss in sis:
            for _ in ll.isomorphic_substructures_iterator(ss):
                good = False
                break
            if not good:
                break
        res.append(good)
    return res

def check_containment_cert(smalls, large):
    sis = [IncidenceStructure(ss.size(), ss.blocks()['edges']) for ss in smalls]
    lis = IncidenceStructure(large.size(), large.blocks()['edges'])
    for ii, ss in enumerate(sis):
        for xx in lis.isomorphic_substructures_iterator(ss):
            return [xx, smalls[ii]]
    return None

# This is some hack to create the theory for 3-graphs without C5- and K4-
# up to size 7. It is easier to make them as extensions of 6 sized structures
# so this code does that.

# Reset three graphs, so nothing is excluded
TG = ThreeGraphTheory
TG.exclude()

# C5 minus
F5 = TG(5, edges=[[0, 1, 2], [0, 3, 4], [1, 3, 4]])

# flags of size 5
fl5 = TG.generate(5)

# boolean vector indicating each element in fl5 if it has C5m
gs = check_containment([F5], fl5)

# k4 and k4m (the two induced structures with size 4 excluded)
k4 = TG(4, edges=[[0, 1, 2], [0, 1, 3], [0, 2, 3], [1, 2, 3]])
k4m = TG(4, edges=[[0, 1, 2], [0, 1, 3], [0, 2, 3]])

# set the excluded structures. k4, k4m and all in fl5 containing C5m
exls = [k4, k4m] + [xx for ii, xx in enumerate(fl5) if not gs[ii]]
TG.exclude(exls)

# check the list of flags with size 5 and 6
fl4 = TG.generate(4)
fl5 = TG.generate(5)
fl6 = TG.generate(6)

# quick 3-graph identifier code. This will be the identifier for
# the theory of C5- free 3-graphs (any identifier working for 3-graphs can work here)
def _identify_hypergraph(n, ftype_points, edges):
    g = Graph([list(range(n+len(edges))), [(i+n,x) for i,b in enumerate(edges) for x in b]], 
              format='vertices_and_edges')
    ftype_union = [jj for ff in ftype_points for jj in ff]
    partt = list(ftype_points) + \
            [[ii for ii in range(n) if ii not in ftype_union]] + \
            [list(range(n,n+len(edges)))]
    blocks = tuple(g.canonical_label(partition=partt).edges(labels=None, sort=True))
    return (n, tuple([len(xx) for xx in ftype_points]), blocks)

# generator code. It should really just return TG, but for size 7 that takes too long
# so this hack just returns TG for size up to 6, and for 7 it generates all flags
# with this extension technique
def _gen(n):
    if n<=4:
        for xx in TG.generate_flags(n):
            yield xx.blocks()
    elif n==5:
        for xx in fl5:
            yield xx.blocks()
    elif n==6:
        for xx in fl6:
            yield xx.blocks()
    elif n==7:
        import itertools
        from tqdm import tqdm
        fl7_m = [[] for ii in range(35+1)]
        subs = list(itertools.combinations(range(6), int(2)))
        for xx in tqdm(fl6):
            xb = xx.blocks()['edges']
            for ii in range(15+1):
                for pps in itertools.combinations(subs, int(ii)):
                    xbp = [[pp[0], pp[1], 6] for pp in pps] + xb
                    flxp = TG(7, edges=xbp)
                    en = len(xbp)
                    if flxp not in fl7_m[en]:
                        if check_containment(exls, [flxp])[0]:
                            fl7_m[en].append(flxp)
        fl7 = [yy for xx in fl7_m for yy in xx]
        for xx in fl7:
            yield xx.blocks()
    else:
        #for n>=8 just return an empty list, this will not be called so doesn't 
        #really matter
        return []

# Create the theory based on this generator and identifier
TGp = CombinatorialTheory("NoF5", _gen, _identify_hypergraph, edges=3)

# for sanity check, print the number of structures with size 5, 6, 7
print(len(TGp.generate(5)), len(TGp.generate(6)), len(TGp.generate(7)))

100%|███████████████████████████████████████████| 30/30 [08:25<00:00, 16.85s/it]

7 30 201


In [1]:
def _identify_23graph(n, ftype_points, edges2, edges3):
    g = Graph([list(range(n+len(edges3))), list(edges2)+[(i+n,x) for i,b in enumerate(edges3) for x in b]], 
              format='vertices_and_edges')
    ftype_union = [jj for ff in ftype_points for jj in ff]
    partt = list(ftype_points) + \
            [[ii for ii in range(n) if ii not in ftype_union]] + \
            [list(range(n,n+len(edges3)))]
    blocks = tuple(g.canonical_label(partition=partt).edges(labels=None, sort=True))
    return (n, tuple([len(xx) for xx in ftype_points]), blocks)

import itertools
from tqdm import tqdm

def _generate_23graph(n):
    fln = TGp.generate(n)
    uniques = []
    for xx in tqdm(fln):
        eds3 = xx.blocks()['edges']
        shadow = []
        for ed in eds3:
            for pair in itertools.combinations(ed, int(2)):
                if pair not in shadow:
                    shadow.append(pair)
        non_shadow = [pair for pair in itertools.combinations(range(n), int(2)) if pair not in shadow]
        for ii in range(len(non_shadow)+1):
            for non_shadow_add in itertools.combinations(non_shadow, int(ii)):
                eds2 = list(non_shadow_add) + shadow
                unii = _identify_23graph(n, [], eds2, eds3)
                if unii not in uniques:
                    uniques.append(unii)
                    yield {'edges2': eds2, 'edges3':eds3}

TT = CombinatorialTheory("NoF5Shadow", _generate_23graph, _identify_23graph, edges2=2, edges3=3)

In [22]:
TT.exclude(TT(4, edges2=[[0, 1], [0, 2], [1, 2], [0, 3], [1, 3], [2, 3]], edges3=[[0, 1, 2], [0, 1, 3]]))

pointed_3edge = TT(3, ftype=[0], edges2=[[0, 1], [0, 2], [1, 2]], edges3=[[0, 1, 2]])
double_pointed_3edge = TT(3, ftype=[0, 1], edges2=[[0, 1], [0, 2], [1, 2]], edges3=[[0, 1, 2]])

poses = [pointed_3edge - 8/45]

k4s = 1
for xx in TT.generate(4):
    if len(xx.blocks()['edges2'])==6:
        k4s += xx
k4s -= 1

#for maximizing k4s
res = TT.optimize(k4s, 6, positives=poses, file="default")

Base flags generated, their number is 804
The relevant ftypes are constructed, their number is 18
Block sizes before symmetric/asymmetric change is applied: [27, 37, 16, 20, 26, 25, 35, 28, 34, 37, 47, 42, 43, 22, 28, 32, 34, 27]


Done with mult table for Ftype on 4 points with edges2=[[0, 1], [0, 2], [1, 2], [0, 3], [1, 3]], edges3=[[0, 1, 2], [0, 1, 3]]: : 18it [00:00, 993.78it/s]


Tables finished


Done with positivity constraint 0: 100%|██████████| 1/1 [00:00<00:00,  3.04it/s]


Constraints finished
Running sdp without construction. Used block sizes are [18, 9, 25, 12, 5, 11, 12, 8, 18, 8, 10, 15, 14, 21, 12, 16, 21, 13, 26, 11, 14, 33, 19, 23, 9, 34, 10, 12, 20, 8, 21, 11, 13, 21, 13, 14, -804, -34]
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 8.74e-01 Pobj: -4.1194521e+01 Ad: 4.88e-01 Dobj: -2.9681491e+01 
Iter:  2 Ap: 1.00e+00 Pobj: -5.4590759e+01 Ad: 8.90e-01 Dobj: -1.4034992e+00 
Iter:  3 Ap: 1.00e+00 Pobj: -5.7056320e+01 Ad: 8.98e-01 Dobj: -4.1854817e-01 
Iter:  4 Ap: 1.00e+00 Pobj: -5.9600770e+01 Ad: 8.17e-01 Dobj: -1.6357452e-01 
Iter:  5 Ap: 7.46e-01 Pobj: -6.4739036e+01 Ad: 6.36e-01 Dobj: -9.3092122e-02 
Iter:  6 Ap: 4.38e-01 Pobj: -4.7951412e+01 Ad: 5.17e-01 Dobj: -6.2489678e-02 
Iter:  7 Ap: 6.12e-01 Pobj: -3.9743977e+01 Ad: 6.06e-01 Dobj: -2.9464695e-02 
Iter:  8 Ap: 6.73e-01 Pobj: -3.0877385e+01 Ad: 5.55e-01 Dobj: -1.8121796e-02 
Iter:  9 Ap: 5.07e-01 Pobj: -2.7702454e+01 Ad: 5.17e-01 Dobj:

In [23]:
#for maximizing edges
edge = TT(2, edges2=[[0, 1]])
res = TT.optimize(edge, 6, positives=poses, file="default")

Base flags generated, their number is 804
The relevant ftypes are constructed, their number is 18
Block sizes before symmetric/asymmetric change is applied: [27, 37, 16, 20, 26, 25, 35, 28, 34, 37, 47, 42, 43, 22, 28, 32, 34, 27]


Done with mult table for Ftype on 4 points with edges2=[[0, 1], [0, 2], [1, 2], [0, 3], [1, 3]], edges3=[[0, 1, 2], [0, 1, 3]]: : 18it [00:00, 898.52it/s]


Tables finished


Done with positivity constraint 0: 100%|██████████| 1/1 [00:00<00:00,  2.91it/s]


Constraints finished
Running sdp without construction. Used block sizes are [18, 9, 25, 12, 5, 11, 12, 8, 18, 8, 10, 15, 14, 21, 12, 16, 21, 13, 26, 11, 14, 33, 19, 23, 9, 34, 10, 12, 20, 8, 21, 11, 13, 21, 13, 14, -804, -34]
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 8.65e-01 Pobj: -4.1447106e+01 Ad: 4.97e-01 Dobj: -9.1762574e+00 
Iter:  2 Ap: 1.00e+00 Pobj: -5.5741527e+01 Ad: 8.83e-01 Dobj:  1.5922207e+00 
Iter:  3 Ap: 1.00e+00 Pobj: -5.8194397e+01 Ad: 9.01e-01 Dobj: -5.8789360e-01 
Iter:  4 Ap: 1.00e+00 Pobj: -6.0579680e+01 Ad: 8.21e-01 Dobj: -6.2466781e-01 
Iter:  5 Ap: 7.35e-01 Pobj: -6.5339916e+01 Ad: 6.38e-01 Dobj: -6.2496412e-01 
Iter:  6 Ap: 4.37e-01 Pobj: -4.8614315e+01 Ad: 5.17e-01 Dobj: -6.3131660e-01 
Iter:  7 Ap: 6.17e-01 Pobj: -4.0166060e+01 Ad: 6.06e-01 Dobj: -6.4218803e-01 
Iter:  8 Ap: 6.75e-01 Pobj: -3.1354070e+01 Ad: 5.54e-01 Dobj: -6.4491148e-01 
Iter:  9 Ap: 5.10e-01 Pobj: -2.8230877e+01 Ad: 5.21e-01 Dobj:

In [21]:
#This cell just prints what the computer things are the probabilities in the optimum
phi_vals = res['phi vectors'][0]
alg = FlagAlgebra(RR, TT)
phi6 = alg(6, phi_vals)
phi4_vals = [phi6.density(xx) for xx in TT.generate_flags(4)]
phi4 = alg(4, phi4_vals)
phi4

Flag Algebra Element over Real Field with 53 bits of precision
0.0371886165441148   - Flag on 4 points, ftype from [] with edges2=[], edges3=[]
0.00449681371497380  - Flag on 4 points, ftype from [] with edges2=[[0, 1]], edges3=[]
0.0597340867310077   - Flag on 4 points, ftype from [] with edges2=[[0, 1], [0, 2]], edges3=[]
0.00239420703452276  - Flag on 4 points, ftype from [] with edges2=[[0, 1], [2, 3]], edges3=[]
0.240651949330466    - Flag on 4 points, ftype from [] with edges2=[[0, 1], [0, 2], [0, 3]], edges3=[]
0.0626784255511573   - Flag on 4 points, ftype from [] with edges2=[[0, 1], [0, 2], [1, 3]], edges3=[]
0.188280126469677    - Flag on 4 points, ftype from [] with edges2=[[0, 1], [0, 2], [1, 3], [2, 3]], edges3=[]
0.00417455002688870  - Flag on 4 points, ftype from [] with edges2=[[0, 1], [0, 2], [1, 2]], edges3=[[0, 1, 2]]
0.0938569313589682   - Flag on 4 points, ftype from [] with edges2=[[0, 3], [0, 1], [0, 2], [1, 2]], edges3=[[0, 1, 2]]
0.306544284166627    - Flag on